In [1]:
# requests 불러오기
# requests = 브라우저의 역할을 대신 해줌(사이트의 요청과 응답 정보를 처리)
import requests as req

In [9]:
# 1. 네이버 url을 통해서 사이트 요청
# req.get("url") : 해당 url로 요청을 보내주고 나서, 응답을 받음!
# Response 200 : 통신에 성공 → 응답을 잘 받았다!
# Response 400 : 클라이언트 오류 → 우리의 실수!
# Response 500 : 서버 오류 → 우리의 잘못은 없다!
res = req.get("https://news.naver.com/")
res
# HTTP 상태를 확인할 수 있는 웹사이트
# https://developer.mozilla.org/ko/docs/Web/HTTP/Status

<Response [200]>

In [10]:
# 응답 객체 전부를 받아왔다!
res.text

'\n<!doctype html>\n<html lang="ko">\n\t<head>\n\t\t<title id="browserTitleArea">네이버 뉴스</title>\n\t\t\n\n\n<script>\n\tfunction isMobileDevice() {\n\t\treturn /^.*(iPhone|iPod|iPad|Android).*/.test(navigator.userAgent);\n\t}\n</script>\n<script>\n\t(function () {\n\t\ttry {\n\t\t\tif (isMobileDevice() && isAbleApplyPrefersColorScheme()) {\n\t\t\t\t\n\t\t\t\tdocument.querySelector("html").classList.add("DARK_THEME");\n\t\t\t}\n\t\t} catch(e) {}\n\n\t\tfunction isAbleApplyPrefersColorScheme() {\n\t\t\t\n\t\t\tif (window.matchMedia("(prefers-color-scheme)").matches === false) {\n\t\t\t\treturn false;\n\t\t\t}\n\n\t\t\tvar userAgent = navigator.userAgent;\n\n\t\t\tif (userAgent.indexOf("NAVER") > -1) {\n\t\t\t\t\n\t\t\t\tif (/.*NAVER\\([a-zA-Z]*;\\s[a-zA-Z]*;\\s([0-9]*);/.test(userAgent)) {\n\t\t\t\t\treturn Number(RegExp.$1) >= 1000;\n\t\t\t\t}\n\t\t\t} else {\n\t\t\t\t\n\t\t\t\treturn document.cookie.indexOf("NSCS=1") > -1;\n\t\t\t}\n\n\t\t\treturn false;\n\t\t}\n\t})();\n</script>\n\n\t

### beautifulsoup 라이브러리
 - 응답받은 글자 정보를 태그별로 파싱해줌!
 - 파싱 : 태그별로 쪼개준다!!

In [11]:
from bs4 import BeautifulSoup as bs

In [12]:
# 받아온 응답 글자 정보들을 html 정보로 바꿔줘!
# bs(응답받은 문자열 데이터, "파싱 방법")

soup = bs(res.text , 'lxml')
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<title id="browserTitleArea">네이버 뉴스</title>
<script>
	function isMobileDevice() {
		return /^.*(iPhone|iPod|iPad|Android).*/.test(navigator.userAgent);
	}
</script>
<script>
	(function () {
		try {
			if (isMobileDevice() && isAbleApplyPrefersColorScheme()) {
				
				document.querySelector("html").classList.add("DARK_THEME");
			}
		} catch(e) {}

		function isAbleApplyPrefersColorScheme() {
			
			if (window.matchMedia("(prefers-color-scheme)").matches === false) {
				return false;
			}

			var userAgent = navigator.userAgent;

			if (userAgent.indexOf("NAVER") > -1) {
				
				if (/.*NAVER\([a-zA-Z]*;\s[a-zA-Z]*;\s([0-9]*);/.test(userAgent)) {
					return Number(RegExp.$1) >= 1000;
				}
			} else {
				
				return document.cookie.indexOf("NSCS=1") > -1;
			}

			return false;
		}
	})();
</script>
<script>
		var g_ssc = 'news.v3_media' || null;
		</script>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,maximum-sca

In [17]:
# 상단의 카테고리 이름들을 가져와보자!
# html 선택자를 활용해서 지정이 가능!
# select(선택자 경로) : 리스트 형으로 반환
# select_one(선택자 경로) : 하나만 반환
top_title = soup.select('ul > li > a > span')

In [15]:
soup.select_one('ul > li > a > span')

<span class="Nitem_link_menu">언론사별</span>

In [23]:
# 태그 안에 content를 지정하기 위해서는 태그명을 제거하는 과정
# → 인덱싱 + text 속성으로 작업

news_item = []
for i in top_title : 
    news_item.append(i.text)
news_item

['언론사별',
 '정치',
 '경제',
 '사회',
 '생활/문화',
 'IT/과학',
 '세계',
 '랭킹',
 '신문보기',
 '오피니언',
 'TV',
 '팩트체크',
 '알고리즘 안내',
 '정정보도 모음']

In [24]:
# 리스트 컴프리헨션 : append()는 정말 많이 쓰이는 문법이다 보니
# 파이썬에서는 이를 한 줄로 작성하게끔 문법을 제공
news_item2 = [i.text for i in top_title]
news_item2

['언론사별',
 '정치',
 '경제',
 '사회',
 '생활/문화',
 'IT/과학',
 '세계',
 '랭킹',
 '신문보기',
 '오피니언',
 'TV',
 '팩트체크',
 '알고리즘 안내',
 '정정보도 모음']

### 실습! 뉴스 타이틀을 수집해보자!
 - 네이버에서 <b>"눈"</b> 검색 후 나오는 뉴스 <b>타이틀(제목)을 수집해보자!</br>

In [27]:
# 1. 서버에 url 값으로 정보를 넘겨 주고, 응답 받기
# 2. 응답받은 데이터 정보를 BeautifulSoup으로 파싱하기
# 3. select를 활영하여 class 또는 태그명으로 접근
# 4. for문을 활용하여 list에 태그명 제거 후 append

res = req.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EB%88%88")
res

soup = bs(res.text , 'lxml')
soup

title = soup.select("a.news_tit")

news_item3= [i.text for i in title]
news_item3

['"빗자루로 내 외제차 쓸어준 주민…정말 허무합니다"',
 '50㎝ 쌓일 ‘눈 지옥’ 나오나... 대설 위기경보 ‘경계’로 상향',
 "HD현대, 최첨단 '눈에 보이는 조선소' 구축 끝냈다",
 "'영하 20도' 올겨울 '최강 한파'…서해·제주 '눈 펑펑'(종합)[오늘날씨]",
 "콜로라도주 판결 나비효과는…'태풍의 눈' 된 美 연방대법원",
 '제주 한라산 눈 44㎝ 쌓여…1100도로·516도로 통제',
 '엑소 ‘첫 눈’ 10년 역주행 1위',
 '충남서해안·전라·제주 많은 눈…아침 최저기온 영하 20도',
 '6.3㎝ 내린 눈에 호남고속도로서 10중 추돌…1명 심정지',
 '올해는 화이트 크리스마스? 25일 중부지방 눈 내릴 수도']